# Part 3, Multilayer Perceptron Neural Network

In [30]:
import tensorflow as tf
import pandas as pd
import numpy as np
import sys
import h5py
import pickle
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from scipy.sparse import csr_matrix, hstack
from sklearn.cross_validation import KFold, train_test_split
from keras.models import Sequential
from keras.models import save_model, load_model
from keras.layers.advanced_activations import PReLU
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.callbacks import EarlyStopping
import matplotlib.patches as mpatches
import time

%matplotlib inline

In [31]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
'''gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))'''
K.set_session(sess)

In [32]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [33]:
train = pd.read_csv('train.csv')

cat_names = [c for c in train.columns if 'cat' in c]

train = pd.get_dummies(data=train, columns=cat_names)

features = [x for x in train.columns if x not in ['id','loss']]

train_x = np.array(train[features])

ntrain = train_x.shape[0]

train_y = np.log(train['loss'] + 200)

print (train_x.shape)
print (train_y.shape)
train_x.shape, train_y.shape

(188318, 1153)
(188318,)


((188318, 1153), (188318,))

In [48]:
xtr, xte, ytr, yte = train_test_split(train_x, train_y, test_size=0.25, random_state=31337)

<img src="http://cdn.rawgit.com/dnkirill/allstate_capstone/master/images/mlp3.svg"></td>

In [61]:
import keras.backend as K
from sklearn.metrics import mean_absolute_error, make_scorer

def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(preds), np.exp(labels))

def xg_eval_mae(yhat, dtrain):
    y = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(y), np.exp(yhat))

def mae_score(y_true, y_pred):
    return mean_absolute_error(np.exp(y_true), np.exp(y_pred))

mae_scorer = make_scorer(mae_score, greater_is_better=False)


def mean_pred(y_true, y_pred):
    return K.mean(K.abs(K.exp(y_true) - K.exp(y_pred)))

def mean_pred_metrics(y_true, y_pred):
    return K.mean(K.abs(K.exp(y_true) - K.exp(y_pred)))


In [73]:
def cross_validate_mlp(mlp_func, nfolds=3):
    folds = KFold(len(train_y), n_folds=nfolds, shuffle=True, random_state = 31337)
    val_scores = np.zeros((nfolds,))
    for k,(train_index, test_index) in enumerate(folds):
        xtr = train_x[train_index]
        ytr = train_y[train_index]
        xte = train_x[test_index]
        yte = train_y[test_index]
        mlp = mlp_func()
        early_stopping = EarlyStopping(monitor='val_loss', patience=10)
        fit = mlp.fit(xtr, ytr, validation_split=0.2, batch_size=128, 
                      nb_epoch=100, verbose=1, callbacks=[early_stopping])
        pred = mlp.predict(xte, batch_size=256)
        score = mean_absolute_error(yte, pred)
        val_scores[k] += score
        print ('Fold {}, MAE: {}'.format(k, score))
    avg_score = np.sum(val_scores) / float(nfolds)
    print ('{}-fold CV score: {}'.format(nfolds, avg_score))
    return avg_score

In [74]:
def hyper_model():
    model = Sequential()
    model.add(Dense(351, input_dim=train_x.shape[1], init='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.578947))
    
    model.add(Dense(293, init='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.26666))
    
    model.add(Dense(46, init='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.188888))
    
    model.add(Dense(1, init='he_normal'))
    model.compile(optimizer='adam',
              loss=mean_pred,metrics=[mean_pred_metrics])
    return model

In [75]:
cv_score = cross_validate_mlp(hyper_model)
("CV score for the final model:", cv_score)

Train on 100436 samples, validate on 25109 samples
Epoch 1/100
100436/100436 [==============================] - 9s - loss: 3210.1322 - mean_pred_metrics: 3210.1322 - val_loss: 3095.4986 - val_mean_pred_metrics: 3095.4986
Epoch 2/100
100436/100436 [==============================] - 7s - loss: 2724.3428 - mean_pred_metrics: 2724.3428 - val_loss: 1932.3458 - val_mean_pred_metrics: 1932.3458
Epoch 3/100
100436/100436 [==============================] - 7s - loss: 2205.4983 - mean_pred_metrics: 2205.4983 - val_loss: 1854.8780 - val_mean_pred_metrics: 1854.8780
Epoch 4/100
100436/100436 [==============================] - 7s - loss: 2081.3217 - mean_pred_metrics: 2081.3217 - val_loss: 1695.6310 - val_mean_pred_metrics: 1695.6310
Epoch 5/100
100436/100436 [==============================] - 8s - loss: 1942.1458 - mean_pred_metrics: 1942.1458 - val_loss: 1565.2848 - val_mean_pred_metrics: 1565.2848
Epoch 6/100
100436/100436 [==============================] - 7s - loss: 1860.2303 - mean_pred_metri

100436/100436 [==============================] - 7s - loss: 1136.6872 - mean_pred_metrics: 1136.6872 - val_loss: 1151.2071 - val_mean_pred_metrics: 1151.2071
Epoch 49/100
100436/100436 [==============================] - 7s - loss: 1125.8715 - mean_pred_metrics: 1125.8715 - val_loss: 1160.0568 - val_mean_pred_metrics: 1160.0568
Epoch 50/100
100436/100436 [==============================] - 7s - loss: 1124.8675 - mean_pred_metrics: 1124.8675 - val_loss: 1157.1427 - val_mean_pred_metrics: 1157.1427
Epoch 51/100
100436/100436 [==============================] - 7s - loss: 1116.8743 - mean_pred_metrics: 1116.8743 - val_loss: 1147.1448 - val_mean_pred_metrics: 1147.1448
Fold 0, MAE: 0.37658946331957105
Train on 100436 samples, validate on 25109 samples
Epoch 1/100
100436/100436 [==============================] - 9s - loss: 3213.2251 - mean_pred_metrics: 3213.2251 - val_loss: 3130.6405 - val_mean_pred_metrics: 3130.6405
Epoch 2/100
100436/100436 [==============================] - 7s - loss: 273

100436/100436 [==============================] - 6s - loss: 1162.7145 - mean_pred_metrics: 1162.7145 - val_loss: 1141.0320 - val_mean_pred_metrics: 1141.0320
Epoch 45/100
100436/100436 [==============================] - 6s - loss: 1152.8343 - mean_pred_metrics: 1152.8343 - val_loss: 1141.9597 - val_mean_pred_metrics: 1141.9597
Epoch 46/100
100436/100436 [==============================] - 6s - loss: 1146.4018 - mean_pred_metrics: 1146.4018 - val_loss: 1143.4258 - val_mean_pred_metrics: 1143.4258
Epoch 47/100
100436/100436 [==============================] - 6s - loss: 1135.1554 - mean_pred_metrics: 1135.1554 - val_loss: 1146.0390 - val_mean_pred_metrics: 1146.0390
Epoch 48/100
100436/100436 [==============================] - 6s - loss: 1133.1676 - mean_pred_metrics: 1133.1676 - val_loss: 1142.4095 - val_mean_pred_metrics: 1142.4095
Epoch 49/100
100436/100436 [==============================] - 6s - loss: 1127.3076 - mean_pred_metrics: 1127.3076 - val_loss: 1147.7889 - val_mean_pred_metric

100436/100436 [==============================] - 6s - loss: 1148.9452 - mean_pred_metrics: 1148.9452 - val_loss: 1152.0684 - val_mean_pred_metrics: 1152.0684
Epoch 42/100
100436/100436 [==============================] - 6s - loss: 1149.1742 - mean_pred_metrics: 1149.1742 - val_loss: 1152.5814 - val_mean_pred_metrics: 1152.5814
Epoch 43/100
100436/100436 [==============================] - 6s - loss: 1140.7909 - mean_pred_metrics: 1140.7909 - val_loss: 1159.4557 - val_mean_pred_metrics: 1159.4557
Fold 2, MAE: 0.37785441310995893
3-fold CV score: 0.3770100302582653


('CV score for the final model:', 0.37701003025826529)

In [78]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
# define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = [1,1,1,1,1,0,0,0,0,0]
# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, nb_epoch=100, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

[[22, 37], [10, 16], [25, 26], [39, 16], [45], [29], [8, 26], [6, 10], [8, 16], [47, 25, 37, 36]]
[[22 37  0  0]
 [10 16  0  0]
 [25 26  0  0]
 [39 16  0  0]
 [45  0  0  0]
 [29  0  0  0]
 [ 8 26  0  0]
 [ 6 10  0  0]
 [ 8 16  0  0]
 [47 25 37 36]]
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_3 (Embedding)          (None, 4, 8)          400         embedding_input_3[0][0]          
____________________________________________________________________________________________________
flatten_3 (Flatten)              (None, 32)            0           embedding_3[0][0]                
____________________________________________________________________________________________________
dense_135 (Dense)                (None, 1)             33          flatten_3[0][0]                  
Total params: 433
Trainable params: 433
Non-

Though this model is not adapted for mere 30 epochs of training, nor for 3-fold CV (I used 5-fold on Kaggle), even though this is a single unbagged model which has been cross-validated on three folds only, we see a very good score:
`CV = 1150` (your score may vary a little).

By the way, this single model, bagged, 5-fold CVed, scored 1116.28 on Kaggle LB.

As we see, this model is considerably better than any other models we had so far. We now take it as the second part of our final ensemble.